In [ ]:
import torch
import pandas as pd
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
import json
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import texthero as hero
from texthero import preprocessing

In [ ]:
text_data = pd.read_json(r"/home/shah/Desktop/FB-Marketplace-Recommendation-Ranking-System/data/products_table.json")

In [ ]:
text_data['product_description']= hero.clean(text_data['product_description'])

In [ ]:
text_data

In [ ]:
custom_pipeline = [preprocessing.fillna,
                   #preprocessing.lowercase,
                   preprocessing.remove_whitespace,
                   preprocessing.remove_diacritics
                   #preprocessing.remove_brackets
                  ]
text_data['product_description'] = hero.clean(text_data['product_description'], custom_pipeline)
text_data['product_description'] = [n.replace('{','') for n in text_data['product_description']]

In [ ]:
text_data.drop(['product_name', 'price', 'location',
       'page_id', 'create_time', 'category'], axis=1, inplace=False)

In [ ]:
def remove_n_a_rows(df, column: str):
    # Swap N/A for the pandas nan, so we can drop them
    temp_df = df[column].replace('N/A', np.nan)
    temp_df = temp_df.dropna()
    # Create a new df with only the records without the nans
    clean_df = pd.merge(temp_df, df,
                            left_index=True, right_index=True)
    # The merge creates a duplicate column. Remove it.
    clean_df.drop(column + '_x', inplace=True, axis=1)
    # Rename the remaining category column
    clean_df.rename(columns={column + '_y': column}, inplace=True)
    # Commit the cleansed data to the dataframe
    df = clean_df
    return df

new_text = remove_n_a_rows(new_text, 'product_description')

In [ ]:
new_text = new_text.dropna()
new_text = new_text.reset_index(drop=True)

In [ ]:
new_text

In [ ]:
new_text['product_description'].isnull().sum()

In [ ]:
def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split() 
        input = [word2id[n] for n in word[:-1]]
        target = word2id[word[-1]] 

        input_batch.append(input)
        target_batch.append(target)

    print(input_batch)
    print(target_batch)

In [ ]:
word_list = " ".join(text_data['product_description']).split()
word_list = list(word_list)
word2id = {w: i for i, w in enumerate(word_list)}
id2word = {i: w for i, w in enumerate(word_list)}
n_class = len(word2id)

n_class

In [ ]:
word2id

In [ ]:
#building the model
class NNLM(nn.Module):
   def __init__(self):
       super(NNLM, self).__init__()
       self.embeddings = nn.Embedding(n_class, m) #embedding layer or look up table

       self.hidden1 = nn.Linear(n_step * m, n_hidden, bias=False)
       self.ones = nn.Parameter(torch.ones(n_hidden))
      
       self.hidden2 = nn.Linear(n_hidden, n_class, bias=False)
       self.hidden3 = nn.Linear(n_step * m, n_class, bias=False) #final layer
      
       self.bias = nn.Parameter(torch.ones(n_class))

   def forward(self, X):
       X = self.embeddings(X) # embeddings
       X = X.view(-1, n_step * m) # first layer
       tanh = torch.tanh(self.d + self.hidden1(X)) # tanh layer
       output = self.b + self.hidden3(X) + self.hidden2(tanh) # summing up all the layers with bias
       return output

In [ ]:
n_step = 2
n_hidden = 2
m = 2

In [ ]:
model = NNLM()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
input_batch, target_batch = make_batch(text_data['product_description'])
input_batch = torch.LongTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

In [ ]:
for epoch in range(5000):
    optimizer.zero_grad()
    embeddings, output = model(input_batch)

    # output : [batch_size, n_class], target_batch : [batch_size]
    loss = criterion(output, target_batch)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

# Predict
predict = model(input_batch).data.max(1, keepdim=True)[1]

# Test
print([sen.split()[:2] for sen in new_text['product_description']], '->', [id2word[n.item()] for n in predict.squeeze()])

In [ ]:
#train the model using word2vec